Homework

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import bs4
import pandas as pd
import re

def get_website(url):
    chromedriver_path = r"C:\Users\jeerasak.RUGBYSCHOOL\Desktop\Ultimate_Python\Bootcamp27Class7\chromedriver.exe"
    driver = webdriver.Chrome(executable_path=chromedriver_path)
    driver.get(url)
    return driver

def choose_thai_language(driver, thai_button):
    thai_button_element = driver.find_element(By.XPATH, thai_button)
    return thai_button_element

def close_popup(driver, popup):
    close_popup_element = driver.execute_script(popup)
    return close_popup_element

def search_engine(driver, search_text):
    search_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
    search_element.send_keys(search_text)
    return search_element

def get_source(source):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    all_product = soup.find_all('div',{source:value})
    return all_product

def get_data(data_dict_list):    
    for product in all_product:
        title_element = product.find('div',{'class':'ie3A+n bM+7UW Cve6sh'})
        title_text = title_element.text

        price_element = product.find('div',{'class':'hpDKMN'})
        price_text = price_element.text

        sales_element = product.find('div',{'class':'r6HknA uEPGHT'})
        if sales_element is None:
            sales_text = None
        else:
            sales_text = sales_element.text

        product_link_element = product.find('a')
        product_link = product_link_element.get('href')
        product_full_link = 'https://shopee.co.th'+product_link

        data_dict = dict()
        data_dict['title'] = title_text
        data_dict['price'] = price_text
        data_dict['sales'] = sales_text
        data_dict['link'] = product_full_link
        data_dict_list.append(data_dict)
        
    return data_dict_list

def clean_price(price):
    result = re.findall(r'[0-9.,]+',price)
    if '-' in price:
        original_price = None
        min_price = result[0]
        max_price = result[1]
    elif len(result) == 2:
        original_price = result[0]
        min_price = result[1]
        max_price = result[1]
    elif len(result) == 1:
        original_price = result[0]
        min_price = result[0]
        max_price = result[0]

    price_list = [original_price,min_price,max_price]
 
    price_clean_list = []
    for price in price_list:
        if price is None:
            price_clean_list.append('None')
        else:
            price_clean_list.append(int(price.replace(',','')))
        
    return price_clean_list

url = ('https://shopee.co.th/')
driver = get_website(url)

time.sleep(1)

thai_button = '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button'
choose_thai_button = choose_thai_language(driver, thai_button)
choose_thai_button.click()

time.sleep(1)

popup = "return document.querySelector('shopee-banner-popup-stateful').shadowRoot.querySelector('div.shopee-popup__close-btn')"
close_popup_button = close_popup(driver, popup)
close_popup_button.click()

time.sleep(1)

search_text = 'เก้าอี้ ergonomic'
search = search_engine(driver, search_text)
search.send_keys(Keys.ENTER)
driver.execute_script("document.body.style.zoom='10%'")

time.sleep(5)

source = 'class'
value = 'col-xs-2-4 shopee-search-item-result__item'
all_product = get_source(source)
data_dict_list = []
data_df = pd.DataFrame(get_data(data_dict_list))
price_clean_ser = data_df['price'].apply(clean_price)
price_clean_list = list(price_clean_ser)
price_clean_df = pd.DataFrame(price_clean_list, columns=['original_price','min_price','max_price'])
price_data_df = pd.concat([data_df,price_clean_df],axis=1)
price_data_df

<ipython-input-1-387a7692e964>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver_path)


,title,price,sales,link,original_price,min_price,max_price
0,Ergonomic chair เก้าอี้ เก้าอี้สำนักงาน เก้าอี...,"฿29,000฿6,500",ขายแล้ว 1 ชิ้น,https://shopee.co.th/Ergonomic-chair-เก้าอี้-เ...,29000,6500,6500
1,เซตคู่เพื่อสุขภาพ FULI X8 ErgoChair Pro Series,"฿6,698฿4,699",ขายแล้ว 450 ชิ้น,https://shopee.co.th/เซตคู่เพื่อสุขภาพ-FULI-X8...,6698,4699,4699
2,เก้าอี้ เก้าอี้ทำงาน เก้าอี้สำนักงาน พนักพิงปร...,"฿1,350 - ฿1,550",ขายแล้ว 45 ชิ้น,https://shopee.co.th/เก้าอี้-เก้าอี้ทำงาน-เก้า...,None,1350,1550
3,Elife Ergonomic chair เก้าอี้สำนักงาน เก้าอี้ท...,"฿2,590 - ฿2,980",ขายแล้ว 61 ชิ้น,https://shopee.co.th/Elife-Ergonomic-chair-เก้...,None,2590,2980
4,Woodpanda เก้าอี้ออฟฟิศ เก้าอี้สำนักงาน เก้าอี...,"฿1,299 - ฿1,399",ขายแล้ว 987 ชิ้น,https://shopee.co.th/Woodpanda-เก้าอี้ออฟฟิศ-เ...,None,1299,1399
5,เก้าอี้เพื่อสุขภาพ รุ่น BIONIC เก้าอี้ เก้าอี...,"฿9,990฿4,990",ขายแล้ว 145 ชิ้น,https://shopee.co.th/เก้าอี้เพื่อสุขภาพ-รุ่น-B...,9990,4990,4990
6,Xiaomi Yuemi YM Ergonomic Office Chair เก้าอี้...,"฿7,399 - ฿8,099",ขายแล้ว 75 ชิ้น,https://shopee.co.th/Xiaomi-Yuemi-YM-Ergonomic...,None,7399,8099
7,[sgoole]Ergonomic Office Computer Game Chair S...,฿417฿250,ขายแล้ว 11 ชิ้น,https://shopee.co.th/-sgoole-Ergonomic-Office-...,417,250,250
8,KUCA 2022 รุ่นจำกัด /ชื่อร่วม กำหนดเองทำ เก้าอ...,"฿372 - ฿1,039",ขายแล้ว 72 ชิ้น,https://shopee.co.th/KUCA-2022-รุ่นจำกัด-ชื่อร...,None,372,1039
9,Xiaomi YM ERGONOMIC Office Chair เก้าอี้ทำงาน ...,"฿7,399 - ฿8,099",ขายแล้ว 14 ชิ้น,https://shopee.co.th/Xiaomi-YM-ERGONOMIC-Offic...,None,7399,8099


In [2]:
price_data_df.to_excel('Homework.xlsx')